In [44]:
!pip install langchain text2vec
!pip install faiss-cpu

Looking in indexes: https://pypi.org/simple, https://pip.repos.neuron.amazonaws.com
Looking in indexes: https://pypi.org/simple, https://pip.repos.neuron.amazonaws.com


In [45]:
import json
with open('workflow_asset/catogory_sample.json', 'r') as f:
    category = json.loads(f.read())
    
with open('workflow_asset/catogory_sample_generate.json', 'r') as f:
    category_generate = json.loads(f.read())

In [86]:
import pandas as pd
pd.options.display.max_rows=1000

In [47]:
def generate_sample():
    
    for c, v in category.items():
        for product_sample in v:
            yield c, product_sample

        for product_sample in category_generate[c]:
            yield c, product_sample

In [48]:
df = pd.DataFrame(generate_sample(), columns=['category', 'sample'])
df['cs'] = df['category']+'-'+df['sample']

In [49]:
from langchain.vectorstores import FAISS, Chroma

In [50]:
from text2vec import SentenceModel
model = SentenceModel('shibing624/text2vec-base-chinese')
embeddings = model.encode(df['sample'])

2023-08-09 12:40:05.789 | DEBUG    | text2vec.sentence_model:__init__:76 - Use device: cpu


In [51]:
from typing import Any, Dict, List, Optional
from text2vec import SentenceModel

class EmbeddingModel():
    def __init__(self, model='shibing624/text2vec-base-chinese', **kwargs):

        self.embedding_model = SentenceModel()


    def embed_documents(self, texts: List[str]) -> List[List[float]]:
        """Compute doc embeddings using a Bedrock model.

        Args:
            texts: The list of texts to embed.

        Returns:
            List of embeddings, one for each text.
        """
        return self.embedding_model.encode(texts)

    def embed_query(self, text: str) -> List[float]:
        """Compute query embeddings using a Bedrock model.

        Args:
            text: The text to embed.

        Returns:
            Embeddings for the text.
        """
        return self.embedding_model.encode([text])[0]


In [171]:

with open('collect.json', 'r') as f:
    collect = json.loads(f.read())
def collect2df_row(collect):
    for c, items in collect.items():
        for i in items:
            yield c, i
df_pred = pd.DataFrame(collect2df_row(collect), columns=['category', 'item_name'])

In [172]:
from langchain.schema import Document
from langchain.vectorstores.utils import DistanceStrategy

In [173]:
import statistics

In [174]:
# item_name = '九陽豆漿機'
# k=5

# founds = vdb.similarity_search_with_score(item_name, k=k)
# founds = [f for f, s in founds if s<thresh]
# ret = statistics.mode(map(lambda x: x.metadata['category'], founds))
# # ret

In [175]:
for K in [1, 3, 5, 10]:
    # match_col = 'sample' # sample, category, cs, 
    for match_col in ['sample', 'category', 'cs']:
        
        pcol = f'pred_mode{K}_{match_col}'
        print(pcol)
        
        docs = list(map(lambda x: Document(page_content=x[1][match_col], metadata=x[1]), df.iterrows()))
        vdb = FAISS.from_documents(docs, EmbeddingModel(), normalize_L2=False, distance_strategy=DistanceStrategy.COSINE)

        def classify_meta(item_name):
            # return vdb.similarity_search_with_score(item_name, k=1)[0][0].metadata['category']
            return vdb.similarity_search(item_name, k=1)[0]#.metadata['category']

        def classify(item_name):
            # return vdb.similarity_search_with_score(item_name, k=1)[0][0].metadata['category']
            return vdb.similarity_search(item_name, k=1)[0].metadata['category']

        def classify_mode(item_name, k=10, thresh=250):
            
            founds = vdb.similarity_search_with_score(item_name, k=k)
            if founds:
                founds = [f for f, s in founds if s<thresh]
                
            if not founds:
                return None
            ret = statistics.mode(map(lambda x: x.metadata['category'], founds))
            
            return ret
        
        df_pred[pcol] = df_pred['item_name'].apply(lambda x: classify_mode(x, k=K))

pred_mode1_sample


2023-08-09 13:27:01.368 | DEBUG    | text2vec.sentence_model:__init__:76 - Use device: cpu


pred_mode1_category


2023-08-09 13:27:34.447 | DEBUG    | text2vec.sentence_model:__init__:76 - Use device: cpu


pred_mode1_cs


...

2023-08-09 13:31:55.458 | DEBUG    | text2vec.sentence_model:__init__:76 - Use device: cpu


pred_mode10_category


2023-08-09 13:32:27.837 | DEBUG    | text2vec.sentence_model:__init__:76 - Use device: cpu


pred_mode10_cs


2023-08-09 13:32:55.262 | DEBUG    | text2vec.sentence_model:__init__:76 - Use device: cpu


In [179]:
df_pred.isna().sum()

category                  0
item_name                 0
pred_mode1_sample        37
pred_mode1_category     335
pred_mode1_cs            38
pred_mode3_sample        37
pred_mode3_category     335
pred_mode3_cs            38
pred_mode5_sample        37
pred_mode5_category     335
pred_mode5_cs            38
pred_mode10_sample       37
pred_mode10_category    335
pred_mode10_cs           38
dtype: int64

In [176]:
from sklearn import metrics

In [181]:
for K in [1, 3, 5, 10]:
    # match_col = 'sample' # sample, category, cs, 
    for match_col in ['sample', 'category', 'cs']:
        pcol = f'pred_mode{K}_{match_col}'
        _df_pred = df_pred[df_pred[pcol].notna()]
        acc = metrics.accuracy_score(_df_pred['category'], _df_pred[pcol])
        print(f'mode{K}, {match_col}, {acc}')

mode1, sample, 0.3831041257367387
mode1, category, 0.43601895734597157
mode1, cs, 0.4271653543307087
mode3, sample, 0.3889980353634578
mode3, category, 0.43601895734597157
mode3, cs, 0.43700787401574803
mode5, sample, 0.4066797642436149
mode5, category, 0.43601895734597157
mode5, cs, 0.43503937007874016
mode10, sample, 0.3948919449901768
mode10, category, 0.43601895734597157
mode10, cs, 0.4271653543307087


In [186]:
df_pred

,category,item_name,pred_mode1_sample,pred_mode1_category,pred_mode1_cs,pred_mode3_sample,pred_mode3_category,pred_mode3_cs,pred_mode5_sample,pred_mode5_category,pred_mode5_cs,pred_mode10_sample,pred_mode10_category,pred_mode10_cs
0,三明治,香濃熔岩起司三明治,三明治,三明治,三明治,三明治,三明治,三明治,三明治,三明治,三明治,三明治,三明治,三明治
1,三明治,烤雞生吐司三明治 2023-05-07 edcn-fiis,餅乾,速食調理包,三明治,三明治,速食調理包,三明治,三明治,速食調理包,三明治,三明治,速食調理包,三明治
2,三明治,新東陽三明治火腿400G/包[400g],三明治,三明治,三明治,三明治,三明治,三明治,三明治,三明治,三明治,三明治,三明治,三明治
3,三明治,燻雞佛卡夏,生鮮蔬果/沙拉,速食調理包,雞胸肉,生鮮蔬果/沙拉,速食調理包,雞胸肉,生鮮蔬果/沙拉,速食調理包,雞胸肉,三明治,速食調理包,雞胸肉
4,中式糕點,芋角,米飯類,None,米飯類,米飯類,None,米飯類,米飯類,None,米飯類,米飯類,None,米飯類
5,中式糕點,李家芋頭糕,米飯類,None,米飯類,米飯類,None,米飯類,米飯類,None,米飯類,米飯類,None,米飯類
6,中式糕點,無—御禾坊芋頭酥,米飯類,None,米飯類,米飯類,None,米飯類,地瓜,None,米飯類,地瓜,None,地瓜
7,中式糕點,賀發 桂花糕 糕點 點心 210克,沖泡即食品,None,中式糕點,沖泡即食品,None,中式糕點,甜筒,None,中式糕點,即溶咖啡,None,中式糕點
8,保久乳,保久乳(蘋果),果汁/果醋飲,保久乳,保久乳,果汁/果醋飲,保久乳,保久乳,果汁/果醋飲,保久乳,果汁/果醋飲,果汁/果醋飲,保久乳,果汁/果醋飲
9,保久乳,光泉全脂保久乳,保久乳,保久乳,保久乳,保久乳,保久乳,保久乳,保久乳,保久乳,奶粉,奶粉,保久乳,保久乳


In [183]:
df_pred[df_pred.pred_mode10_cs.isna()]

,category,item_name,pred_mode1_sample,pred_mode1_category,pred_mode1_cs,pred_mode3_sample,pred_mode3_category,pred_mode3_cs,pred_mode5_sample,pred_mode5_category,pred_mode5_cs,pred_mode10_sample,pred_mode10_category,pred_mode10_cs
14,保健/養生,2*活力蛋白素,None,None,None,None,None,None,None,None,None,None,None,None
18,個人服飾配件,中古機車斜背波士頓小包,行李箱/配件,None,None,行李箱/配件,None,None,行李箱/配件,None,None,行李箱/配件,None,None
41,其他甜食,法式布蕾派,麵包,None,None,麵包,None,None,麵包,None,None,麵包,None,None
43,其他甜食,愛心圈圈,None,None,None,None,None,None,None,None,None,None,None,None
124,威士忌,約翰走路(綠牌),None,None,None,None,None,None,None,None,None,None,None,None
136,家飾,【Loviisa 英文字母】無痕壁貼 壁紙 除舊佈新,None,None,None,None,None,None,None,None,None,None,None,None
147,小菜/滷味,黃金雞柳條8條,漢堡,None,None,漢堡,None,None,漢堡,None,None,漢堡,None,None
156,布丁/果凍,統一大布丁 180g*3/組#816894,沖泡類飲品,None,None,沖泡類飲品,None,None,沖泡類飲品,None,None,沖泡類飲品,None,None
160,床墊,雙人經濟C,None,None,None,None,None,None,None,None,None,None,None,None
178,手錶,【SEIKO 精工】Lukia系列 綻放 太陽能腕錶(SUT,None,None,None,None,None,None,None,None,None,None,None,None


In [185]:
print(metrics.classification_report(_df_pred['category'], _df_pred[pcol]))

              precision    recall  f1-score   support

         三明治       0.50      0.75      0.60         4
        中式糕點       0.50      0.25      0.33         4
       仙草/愛玉       0.33      1.00      0.50         1
         保久乳       0.33      0.50      0.40         4
       保健/養生       0.00      0.00      0.00         3
      個人服飾配件       0.11      0.67      0.18         3
          傢俱       0.67      0.50      0.57         4
          優格       0.25      0.25      0.25         4
         優酪乳       0.50      0.25      0.33         4
        其他冰品       0.00      0.00      0.00         1
      其他冷凍商品       0.00      0.00      0.00         1
      其他寵物食品       0.00      0.00      0.00         4
      其他日用百貨       0.00      0.00      0.00         1
      其他民生食材       0.00      0.00      0.00         1
        其他甜食       0.00      0.00      0.00         2
      其他生鮮食品       0.00      0.00      0.00         1
          冰塊       0.50      1.00      0.67         1
       冰棒/雪糕       0.67    

/home/ec2-user/anaconda3/envs/pytorch_p310/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/ec2-user/anaconda3/envs/pytorch_p310/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/ec2-user/anaconda3/envs/pytorch_p310/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, mo